<a href="https://colab.research.google.com/github/synthesis0x42/SVGD-QSE/blob/main/PyMC_tomography_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pymc

In [6]:
import pymc as pm
import numpy as np

import plotly.graph_objects as go

In [7]:
sampled_xyz = np.random.uniform(low=-1, high=1, size=3)
while(np.sum(sampled_xyz**2)>1):
    sampled_xyz = np.random.uniform(low=-1, high=1, size=3)
  # Sample one set of biases

theta = (1 + sampled_xyz) / 2

coin_choice_fixed = np.random.choice([0, 1, 2], size=100, p=np.ones(3) / 3)

data = np.random.binomial(n=1, p=theta[coin_choice_fixed])  # n=1 for Bernoulli trials

In [8]:
sampled_xyz

array([-0.41043411, -0.79850148,  0.10164328])

In [9]:
coin_choice_fixed

array([2, 0, 2, 2, 0, 1, 2, 1, 2, 2, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 2, 2,
       2, 1, 0, 2, 1, 1, 0, 0, 2, 2, 1, 0, 1, 0, 0, 1, 1, 1, 2, 0, 2, 2,
       1, 1, 0, 2, 2, 0, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 0, 2, 2, 1, 2, 0,
       2, 1, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 2,
       2, 2, 0, 1, 1, 0, 0, 2, 0, 2, 0, 2])

In [6]:
data

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

In [10]:
with pm.Model() as model:
    # Prior (Uniform on cube)

    xyz = pm.Uniform('xyz', lower=-1, upper=1, shape=3)
    # are px py pz not beta distributions?
    # Likelihood
    const = pm.Potential("constraint",pm.math.switch(pm.math.le(pm.math.sum(xyz**2),1),0,-np.inf))
    theta = (1 + xyz) / 2



    likelihood = pm.Bernoulli('likelihood', p=theta[coin_choice_fixed], observed=data)


    # Inference
    approx = pm.fit(method=pm.SVGD(n_particles=160, jitter=1.))
    trace = approx.sample(1000)

Output()

In [11]:
# Extract 'xyz' data using arviz
posterior = trace.posterior
xyz_data = posterior.data_vars['xyz']


# Generate prior samples for plotting (10000 samples for better visualization)
prior_samples = np.random.uniform(low=-1, high=1, size=(1000, 3))

# Visualization (3D Scatterplot with Plotly)
fig = go.Figure(data=[
    go.Scatter3d(  # Posterior
        x=xyz_data[0, :, 0],
        y=xyz_data[0, :, 1],
        z=xyz_data[0, :, 2],
        mode='markers',
        marker=dict(size=2, color='blue', opacity=0.25),
        name='Posterior'
    ),
    go.Scatter3d(  # Prior
        x=prior_samples[:, 0],
        y=prior_samples[:, 1],
        z=prior_samples[:, 2],
        mode='markers',
        marker=dict(size=2, color='orange', opacity=0.25),
        name='Prior'
    ),
    go.Scatter3d(  # True state
        x=[sampled_xyz[0]],
        y=[sampled_xyz[1]],
        z=[sampled_xyz[2]],
        mode='markers',
        marker=dict(size=15, color='red', symbol='cross'),  # Star symbol for the true state
        name='True State'
    )
])

fig.update_layout(
    title='Prior, Posterior, and True State',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
    )
)
fig.show()

In [12]:
x=xyz_data[0, :, 0],
y=xyz_data[0, :, 1],
z=xyz_data[0, :, 2]

In [15]:
z

<xarray.DataArray 'xyz' (draw: 1000)> Size: 8kB
array([ 0.21992008,  0.22297819,  0.4157016 ,  0.37464484,  0.35646702,
        0.27151431, -0.18320602, -0.02335958,  0.35646702,  0.39749481,
        0.34827344,  0.4335691 ,  0.47015357,  0.32567189,  0.33801024,
        0.40791177,  0.35646702,  0.34281107,  0.39749481,  0.09187338,
        0.23053177,  0.52419991,  0.32592024,  0.40272091,  0.21943974,
        0.29693387,  0.10777485,  0.45865164,  0.12908826,  0.47015357,
        0.1163464 ,  0.29494293,  0.15881263,  0.22612505,  0.50967306,
        0.12908826,  0.10827867,  0.43062587,  0.53062577,  0.02001189,
        0.41609651,  0.44169149,  0.30763882,  0.65255684,  0.29364519,
        0.09508542,  0.03770079,  0.3872058 ,  0.16225255,  0.48048152,
        0.50734511,  0.26238012,  0.21943974,  0.42806801,  0.10587894,
        0.55986696,  0.29693387,  0.36648124,  0.23680079,  0.1557324 ,
        0.55986696, -0.0074599 ,  0.38077897,  0.31789754,  0.43712014,
        0.60430576,  0.21992008,  0.21943974,  0.12466705,  0.43062587,
        0.49990226,  0.53156211,  0.41898398,  0.26469305,  0.31288885,
        0.51979101,  0.40969788,  0.12466705,  0.39373618,  0.35595063,
        0.4340293 ,  0.32567189,  0.2952466 ,  0.16225255,  0.27151431,
        0.45549171,  0.29494293,  0.42808757,  0.35624913,  0.35853072,
        0.36431939,  0.29364519,  0.40272091,  0.39749481,  0.58082682,
        0.46353043, -0.0074599 ,  0.15881263,  0.52419991,  0.22297819,
...
        0.15047661,  0.36146016,  0.42602509,  0.43181906,  0.61108865,
       -0.0074599 ,  0.23096626,  0.53439946,  0.26600761,  0.10439137,
        0.02001189,  0.42808757,  0.35400238,  0.35098517,  0.06625546,
        0.10777485,  0.43062587,  0.35098517,  0.15047661,  0.15047661,
        0.43181906,  0.3486142 ,  0.35853072,  0.34511566,  0.53062577,
        0.47088787,  0.48607454,  0.34281107,  0.26600761,  0.26238012,
        0.11933312,  0.4340293 ,  0.45384408,  0.45384408,  0.29494293,
        0.21992008,  0.36068928,  0.28809715,  0.32567189,  0.43062587,
        0.51079419,  0.65255684,  0.29494293,  0.32567189,  0.21943974,
       -0.18320602,  0.39373618,  0.12908826,  0.31596606,  0.51979101,
        0.29364519,  0.50734511,  0.22297819,  0.18554361,  0.10587894,
        0.17079976,  0.34827344,  0.23222204,  0.44169149,  0.30748686,
        0.23222204,  0.48048152,  0.27380631,  0.43181906,  0.20473048,
        0.3873294 ,  0.44026609,  0.17944272,  0.22322985,  0.26469305,
        0.21496674,  0.35646702,  0.47088787,  0.30018166,  0.28809715,
        0.33632567,  0.33632567,  0.29494293,  0.39749481,  0.53156211,
        0.36648124,  0.3486142 ,  0.15047661,  0.20473048,  0.10587894,
        0.51156616,  0.21496674,  0.21992008,  0.48048152,  0.31596606,
        0.10439137,  0.34281107,  0.4732059 ,  0.28809715,  0.44026609,
        0.51455503,  0.22612505,  0.52419991,  0.32455884,  0.15047661])
Coordinates:
    chain      int64 8B 0
  * draw       (draw) int64 8kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
    xyz_dim_0  int64 8B 2